In [1]:
import numpy as np
import re
import h5py as h5
import matplotlib.pyplot as plt
import tables as tb
%matplotlib inline
plt.style.use('ggplot')

def load_data(data_path, input_node):
    data = tb.open_file(data_path, 'r')
    X = data.get_node('/' + input_node)[:]
    data.close()
    return X

In [19]:
baseline_dir = '/projects/nikhil/ADNI_prediction/input_datasets/'
naming = 'Exp5'
side = 'l'
#HDF file generate from Jon's scripts comprising matched candidate labels and filenames.
if naming == 'Exp4':
    #PTID_idx = 0 #Filename has only one PTID
    input_file = baseline_dir + 'HC_preproc/combined.h5'

    # Run this script for each split, side combination:
    split = 'test'

    input_HC = load_data(input_file,'{}_{}_data'.format(side,split))
    #input_classes = input_data['{}_{}_classes'.format(side, split)][:]
    input_files = load_data(input_file,'{}_{}_files'.format(side,split))    
    
if naming == 'Exp5':
    PTID_idx = 1 #Filename has two PTIDs
    if side == 'l':
        #input_file = '/projects/jp/NI-ML/preprocessing/adni2/04_mask/candidate_left.h5'
        input_file = '/projects/nikhil/ADNI_prediction/input_datasets/HC_preproc/adni1_and_2/candidate_l_factor=1.0_thresh=300_data.h5'
    else:
        #input_file = '/projects/jp/NI-ML/preprocessing/adni2/04_mask/candidate_right.h5'
        input_file = '/projects/nikhil/ADNI_prediction/input_datasets/HC_preproc/adni1_and_2/candidate_r_factor=1.0_thresh=300_data.h5'
    
    input_HC = load_data(input_file,'data')
    #input_classes = input_data['{}_{}_classes'.format(side, split)][:]
    input_files = load_data(input_file,'files')    
    
id_participant = re.compile(r"""
 (?<=ADNI_)      # Match the first string after ADNI_
 (.*?)          # Lazy quantifier so it only grabs the first immediate match.
 (?=_MR)        # End at the _MR
""", re.VERBOSE)

In [ ]:
# T100_L = 74865./12756
# T100_R = 71631./13610
# T300_L = 74681./11076
# T300_R = 79392./10161
# T500_L = 62600./9732
# T500_R = 66583./8953
# T750_L = 50276./8713
# T750_R = 55167./8039

# print ' T100_L:{} T100_R:{} \n T300_L:{} T300_R:{} \n T500_L:{} T500_R:{} \n T750_L:{} T750_R:{}'.format(T100_L,T100_R,T300_L,T300_R,T500_L,T500_R,T750_L,T750_R)


In [ ]:
#change HC labels to (-1,1) from (0,1)
input_HC = 2*input_HC - 1

In [ ]:
print input_HC.shape
print input_files[:4]
f= input_files[0]
subject_id = re.findall(id_participant, f)
print subject_id
print input_HC[:3,5000:5100]

In [20]:
# Generate dictionary {subject_id:[candidate_labels]}
import collections
import scipy.stats as stats
subject_idx=[]
unique_subject_idx = []
subject_vol_dict = collections.OrderedDict()
subject_class_dict = collections.OrderedDict()
ADNI1_PTID = 0
ADNI2_PTID = 1
#find volume indices for each unique subject
seen = set([])
ADNI1 = False
ADNI2 = True
for j, f in enumerate(input_files):
    #subject_id = re.search(id_participant, f).group(0) 
    
    subject_id_list = re.findall(id_participant, f) #ADNI 2 has two sub_id matches - pick the second one [0,1]. 
    subject_id = ''
    if ADNI1 and len(subject_id_list) == 1:
        subject_id = subject_id_list[ADNI1_PTID] 
    elif ADNI2 and len(subject_id_list) == 2: 
        subject_id = subject_id_list[ADNI2_PTID]
    else:
        subject_id = ''
        
    if not subject_id == '':
        subject_idx.append(subject_id)
        if (subject_id not in subject_vol_dict) and not (np.sum(input_HC[j]) == 0):
            subject_vol_dict[subject_id]=[]
            #subject_class_dict[subject_id]=[]

        # check if the candidate label is not empty
        if not np.sum(input_HC[j]) == 0:
        
            subject_vol_dict[subject_id].append(input_HC[j])
            #subject_class_dict[subject_id].append(input_classes[j])

        if subject_id not in seen:
            unique_subject_idx.append(subject_id)
            seen.add(subject_id)

print "Dictionary length: " + str(len(subject_vol_dict))
    
print len(unique_subject_idx)

Dictionary length: 681
739


In [21]:
#Save Dictionaries
exp_name = 'Exp5'
cohort = 'ADNI2'
side = 'left'

sub_HC_vol_file = baseline_dir + 'HC/subject_HC_vol_dictionary_{}_{}_{}.pkl'.format(cohort,side,exp_name)

import pickle
f = open(sub_HC_vol_file, 'wb')
pickle.dump(subject_vol_dict, f)
f.close()


In [22]:
tmp = subject_vol_dict.values()
matches = [x for x in tmp if x == []]
print len(matches)

0


In [ ]:
#Visualize HC
import random
plt.rcParams['figure.figsize'] = (20, 10)
input_file = '/projects/nikhil/ADNI_prediction/input_datasets/HC_preproc/adni1_and_2/candidate_l_factor=1.0_thresh=300_data.h5'
datamask =  load_data(input_file.format(side),'datamask')
volmask = load_data(input_file.format(side),'volmask')
baseline_shape = volmask.shape
features = load_data(input_file.format(side),'data')

sampx = 20
X = np.array(random.sample(features,sampx))
print X.shape
max_per_feature = np.max(X,axis=0)
print '{} HC shape {}, np.sum(np.max(X,axis=0)): {}, diff={}'.format(side, X.shape,np.sum(max_per_feature),X.shape[1]-np.sum(max_per_feature))

plot_list = []
for i in np.arange(sampx):    
    plot_list.append((X[i], 'X {}'.format(i)))    
    
plot_slices(plot_list, baseline_shape, datamask)

In [ ]:
from scipy.stats.mstats_basic import mquantiles
from sklearn.utils.extmath import randomized_svd

def get3DVol(HC_input, HC_shape, input_mask):
    flatvol = np.zeros(np.prod(HC_shape))
    flatvol[input_mask] = HC_input
    vol = flatvol.reshape(-1, HC_shape[2]).T
    return vol

def plot_slices(slice_list, baseline_shape, baseline_mask, llimit=0.01, ulimit=0.99, xmin=1000, xmax=2400):
    """
    Plot dem slices.
    :param slice_list:
    :param llimit:
    :param ulimit:
    :param num_slices:
    :param xmin:
    :param xmax:
    :return:
    """
    num_slices = len(slice_list)
    plt.style.use('ggplot')
    plt.figure()
    cols = 1
    rows = num_slices / cols
    plt.cla()
    for j, input in enumerate(slice_list):
        quantiles = mquantiles(input[0], [llimit, ulimit])
        wt_vol = get3DVol(input[0], baseline_shape, baseline_mask)
        plt.subplot(rows, cols, j + 1)
        #im = plt.imshow(wt_vol[:, xmin:xmax], cmap=plt.cm.Reds, aspect='auto', interpolation='none', vmin=-.06, vmax=0.06)
        im = plt.imshow(wt_vol, cmap=plt.cm.Reds, aspect='auto', interpolation='none')
        plt.grid()
        plt.title(input[1])
        plt.colorbar()
        im.set_clim(quantiles[0], quantiles[1])
        plt.axis('off')

In [ ]:
f = input_files[-4]
subject_id = re.findall(id_participant, f)
print 

In [ ]:
np.sum(X,axis=1)

In [ ]:
import pickle
sub_DX_data_path = baseline_dir + 'CS/ADNI1_BL_PTID_DX_bl_dict.pkl'
sub_DX_dict = pickle.load( open(sub_DX_data_path, "rb" ) )

X = []
y = []
y_dx = [] #Used for creating balanced k-folds
for key in subject_vol_dict.keys():
    X.append(np.sum(np.squeeze(np.array(stats.mode(subject_vol_dict[key])[0]))))
    y.append(sub_DX_dict[key])
    
X = np.array(X)
y = np.array(y)
print 'X shape: {}, y shape: {}'.format(X.shape, y.shape)

In [ ]:
a=0.7
plt.hist(X[y=='AD'],alpha=a,bins=100,label='AD')
plt.hist(X[y=='CN'],alpha=a,bins=100,label='CN')
plt.hist(X[y=='LMCI'],alpha=a,bins=100,label='MCI')
plt.legend()

In [ ]:
plt.subplot(2,1,1)
plt.hist(R_HC_vols)
plt.title('Right HC')
plt.subplot(2,1,2)
plt.hist(L_HC_vols)
plt.title('Left HC')
#R:12386 #L=10262